# PhoneRecommender

### *Autheurs : Baudet Quentin et Larmaillard-Noiren Joris*


PhoneRecommender est un chatbot de recommandation de smartphones basé sur un pipeline **Retrieval-Augmented Generation (RAG)** et un modèle de langage pré-entraîné.

---

## Contexte et objectifs

Ce projet a pour but de construire un assistant intelligent capable d’aider un utilisateur à choisir un téléphone portable en fonction de critères (prix, performance, autonomie, photo, etc.).
Pour cela, nous exploitons :
- Un **dataset Kaggle** contenant les fiches techniques et les spécifications de centaines de modèles de smartphones.
- Un pipeline **RAG** : on indexe les descriptions produits dans un moteur de similarité (FAISS) puis on combine la récupération de documents pertinents à la génération de texte par un **LLM** (Mistral 7B, GPT-2, LLaMA …, au choix).

---

## Fonctionnalités

1. **Exploration et nettoyage du dataset**
   - Analyse exploratoire (EDAx).
   - Standardisation des noms de champs (RAM, stockage, autonomie…).
2. **Indexation documentaire**
   - Conversion du CSV en fragments textuels.
   - Construction d’un index FAISS pour recherches rapides.
3. **Pipeline RAG**
   - À chaque question utilisateur, récupération des **k fiches les plus pertinentes**.
   - Enrichissement du prompt avec ces extraits.
   - Génération de la réponse via un LLM.
4. **Questions réponses dans le terminal**
   - L'utilisateur obtient sa recommandation de téléphone dans le terminal

 
---

## Exploration du dataset

Avant de construire notre modèle de recommandation de smartphones, il est crucial de bien comprendre les caractéristiques et la structure du dataset fourni. Dans ce notebook, nous réalisons une Analyse Exploratoire des Données (EDA) afin de mieux cerner la distribution des différentes variables, détecter les éventuelles anomalies, valeurs manquantes ou incohérences, et identifier les relations potentielles entre les attributs.

Cette étape nous permettra d’extraire des insights précieux pour orienter le pré-traitement des données et la sélection des fonctionnalités les plus pertinentes, en vue d’optimiser les performances du modèle de recommandation.

### Chargement des données

In [ ]:
### Importation des modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
### Chargement des données
data = pd.read_csv('../data/raw/Smartphones_cleaned_dataset.csv')

data.head()

Dans un premier temps nous allons reformater les prix en euros : les données qui sont présentées ci-dessus viennent du marché des smartphones en Inde. Nous allons donc faire une conversion des prix, ici en roupie, en euros.

In [ ]:
### Conversion des prix roupie -> euro
data['price'] = data['price'] * 0.009966

### Arrondi des prix
data['price'] = data['price'].round(2)

data = data.rename(columns={'price': 'price (€)'})

data.head()

**Dimension du dataset**

In [ ]:
data.shape

**Colonnes du dataset**

In [ ]:
data.columns

Maintenant, nous allons passer par l'étape d'exploration des données.

### EDA

### Importation des modules
import plotly.express as px
import plotly.graph_objects as go

**Colonne des marques**

### Distribution des marques
fig_brand = px.histogram(data, x='brand_name', nbins=20, title="Distribution des marques de téléphones vendus")

fig_brand.update_layout(
    xaxis_title='Marque',
    yaxis_title='Nombre de smartphones',
    bargap=0.1
)

fig_brand.show()

* Marques les plus fréquentes :
    - Xiaomi, OnePlus, et Samsung sont les plus représentées, avec environ 130 modèles ou variantes chacun.
    - Apple, Realme, Motorola, Infinix, Vivo, Oppo et Nokia suivent avec une forte présence.
* Longue traîne de petites marques :
    - De nombreuses marques (plus de 20) sont représentées de façon marginale (moins de 10 modèles), voire parfois très peu présentes.


**Colonnes des prix**

Ici, nous affichons la distribution des prix des smartphones.

In [ ]:
### Distribution des prix
fig_price = px.histogram(data, x='price (€)', nbins=20, title='Distribution des prix des smartphones')

fig_price.update_layout(
    xaxis_title='Prix (€)',
    yaxis_title='Nombre de smartphones',
    bargap=0.1
)

fig_price.show()

* Distribution très asymétrique : la majorité écrasante des smartphones sont concentrés dans la tranche basse des prix, en dessous de 1 000 €.
* Longue queue vers la droite : quelques smartphones très haut de gamme (jusqu’à 6000 €) tirent la distribution vers la droite, mais ils sont extrêmement rares.
* Pointe forte autour de l'entrée de gamme : le pic de fréquence se situe dans les premiers intervalles de prix (probablement entre 100 € et 500 €).

**Prix vs. Capacité de la batterie**

In [ ]:
fig_battery_price = px.scatter(
    data,
    x='battery_capacity',
    y='price (€)',
    color='brand_name',
    title='Prix vs Capacité de batterie des smartphones',
    labels={'battery': 'Batterie (mAh)', 'price': 'Prix (€)'},
    opacity=0.6
)

fig_battery_price.update_layout(
    xaxis_title='Capacité batterie',
    yaxis_title='Prix (€)',
    bargap=0.1
)

fig_battery_price.show()

* Tendance générale :
    - Il n’y a pas de corrélation claire entre le prix et la capacité de batterie.
    - Des téléphones à prix très bas peuvent avoir une batterie très élevée (outliers visibles à droite).
    - Inversement, certains téléphones chers n'ont qu'une capacité moyenne.
* Présence d’outliers :
    - Des smartphones affichent une capacité batterie supérieure à 10 000 mAh, ce qui est anormalement élevé pour des téléphones classiques → probablement des erreurs ou des modèles atypiques (tablettes, rugged phones).
* Dispersion importante :
    - Il y a une forte variabilité des prix pour une même capacité batterie, ce qui montre que la batterie n’est pas un facteur déterminant du prix.

**Comparaison entre le prix et les différentes colonnes**

In [ ]:
### Liste des colonnes à comparer au prix
colonnes = ['rating', 'num_cores', 'processor_speed', 'ram_capacity', 'internal_memory', 'screen_size', 'refresh_rate', ]

### Création des traces pour chaque colonne
fig_compare_to_price = go.Figure()

for i, col in enumerate(colonnes):
    fig_compare_to_price.add_trace(
        go.Scatter(
            x=data[col],
            y=data['price (€)'],
            mode='markers',
            name=col,
            visible=(i == 0)
        )
    )

### Création du menu déroulant
menu = [dict(
    buttons=[dict(
        label=col,
        method='update',
        args=[{'visible': [i == j for j in range(len(colonnes))]},
              {'xaxis': {'title': col}}]
    ) for i, col in enumerate(colonnes)],
    direction='down',
    showactive=True,
    x=0.1,
    y=1.15
)]

fig_compare_to_price.update_layout(
    updatemenus=menu,
    title='Prix vs variable sélectionnée',
    xaxis_title=colonnes[0],
    yaxis_title='Prix (€)',
    height=600
)

fig_compare_to_price.show()

**Rating vs prix**

* On observe une légère tendance croissante : les téléphones avec un meilleur rating tendent à être plus chers, surtout à partir de 75+.
* Cela peut s’expliquer par le fait que les modèles mieux notés sont généralement plus performants ou mieux finis.
* La majorité des téléphones ont une note comprise entre 75 et 90.
* Très peu de modèles en dessous de 70, ce qui peut indiquer un biais dans les avis (seuls les bons modèles sont évalués ou conservés dans le dataset).

**Processor_speed vs prix**

* On observe une corrélation positive modérée : en général, plus la vitesse du processeur augmente, plus les smartphones ont tendance à être chers.
* Beaucoup de modèles se situent entre 2.0 et 3.2 GHz, avec des prix allant de 100 € à 1000 €.
* Cela semble représenter le segment "standard à performant" du marché.
* Plus on monte en vitesse, plus les prix deviennent variables, indiquant que la vitesse du processeur n’est pas le seul facteur déterminant du prix.

**Internal_memory vs prix**

* Il existe une relation positive globale entre la capacité de stockage (en Go) et le prix du smartphone.
* On voit des groupes bien distincts :
    - 32–64 Go : téléphones d’entrée de gamme à prix très accessibles.
    - 128–256 Go : majorité des modèles milieu/haut de gamme.
    - 512–1024 Go (1 To) : modèles premium uniquement.
* À 256 Go, par exemple, les prix vont de ~400 € à plus de 2000 € → cela indique que le stockage ne suffit pas à expliquer le prix à lui seul.
* D’autres caractéristiques comme la RAM, le processeur, la marque ou la caméra jouent un rôle majeur.

**Screen_size vs prix**

* Il existe une légère corrélation positive entre la taille de l’écran et le prix : les téléphones avec un écran plus grand sont en moyenne un peu plus chers.
* Très forte densité autour de 6.3 à 6.7 pouces, avec des prix allant de 100 € à 1500 €.
* Cela suggère que la plupart des smartphones sont conçus pour offrir un compromis entre confort d’écran et ergonomie.
* À taille d’écran égale, le prix peut énormément varier → ce qui confirme que l’écran est un critère secondaire dans la tarification par rapport à :
    - la marque,
    - le processeur,
    - les fonctionnalités avancées (caméras, design, etc.).

**Refresh_rate vs prix**

* On observe une élévation du prix moyen à mesure que le taux de rafraîchissement augmente.
    - 60 Hz → présent sur des modèles de toutes gammes (entrée à premium).
    - 120 Hz et + → associés à des smartphones plus récents ou gaming, donc souvent plus chers.
* À 60 Hz, très grande dispersion de prix (de < 100 € à > 6000 €), ce qui confirme que ce n’est pas un critère déterminant seul.
* À 120–144 Hz, les prix restent modérément élevés avec moins de dispersion : cette technologie est réservée aux gammes moyennes à hautes.
* À 240 Hz, très peu de modèles → probablement des modèles gaming haut de gamme.

---

## Preprocessing des données

Nous allons maintenant procéder à un pré-traitement approfondi des données issues du dataset de smartphones disponible sur Kaggle. L’objectif est de nettoyer, transformer et enrichir ces données afin de construire un jeu de données final, adapté à la conception d’un modèle d’intelligence artificielle dédié à la recommandation de smartphones.

**Suppression de colonnes**

Dans cette section, nous allons supprimer certaines colonnes qui ne nous semblent pas pertinentes, ou déterminantes pour le choix d'un téléphone, notamment des fonctionnalités qui sont gadgets comme `has_ir_blaster`. Cela permettra d'être plus orienté vers la satisfaction de la clientèle.

In [ ]:
### Suppression des colonnes
data = data.drop(columns=['has_ir_blaster', 'has_nfc'])

**Critère d'évaluation de l'autonomie d'un téléphone**

Maintenant, nous allons rajouter une étiquette à chaque téléphone pour évaluer le niveau d'autonomie de chaque téléphone, dans une démarche de satisfaction client. Si un client souhaite un téléphone avec une bonne autonomie, et rien d'autre cela permettra de mieux l'orienter vers ce qu'il recherche.

In [1]:
### Fonction d'étiquetage des téléphones par rapport à leur autonomie
def evaluate_battery_capacity(capacity) -> str:
    """
    Attribue une étiquette à chaque téléphone sur le seul critère de la capacité de leur batterie.
    
    :param capacity: Capacité du téléphone
    :return: Étiquette finale
    """
    
    if capacity < 3500:
        return "faible"
    elif capacity < 4500:
        return "moyenne"
    else:
        return "haute"

In [ ]:
### Application de la fonction
data['quality_battery_autonomy'] = data['battery_capacity'].apply(evaluate_battery_capacity)

**Création d'une fonction de scoring pour catégoriser les téléphones selon leur performance**

Dans la section suivante, nous allons implémenter une fonction dans laquelle chaque ligne du dataset sera analysée, et pour chaque attribut des téléphones, nous allons attribuer un score qui sera utilisé après pour calculer un score final qui déterminera la catégorie du téléphone. Cela permettra d'affiner les futures recommandations du modèle, et de fournir des résultats qui pourront répondre aux besoins du client.

In [ ]:
### Importation module
from typing import List, Tuple

### Fonction qui calcule un score total de chaque téléphone
def compute_total_score(phone) -> float:
    """
    Calcule un score de performance pour un téléphone donné 
    en fonction de ses caractéristiques techniques.

    Ce score permet ensuite de classifier le téléphone 
    en haute performance, performance intermédiaire ou basique.
    :param phone: Correspond au téléphone (ligne) que l'on considère actuellement pour le calcul du score
    :return: Le score final du téléphone en fonction des différents critères d'évaluation
    """
    ### Fonction imbriquée qui généralise la logique de calcul des scores : pour les colonnes numériques
    def score_range(val, seuils: List[Tuple[float, float]]) -> float:
        for seuil, point in seuils:
            if val >= seuil:
                return point
        return 0
    
    ### Fonction imbriquée qui généralise la logique de calcul des scores : pour les colonnes booléennes
    def score_boolean(val, point_if_true=0.5) -> float:
        return point_if_true if val else 0
    
    ### Variable qui stocke le score final
    score: float = 0
    
    ### Poids de chaque caractéristique des téléphones attribués selon l'importance de la caractéristique pour le choix du téléphone
    weights: List[float] = [0.10, 0.10, 0.20, 0.10, 0.15, 0.10, 0.10, 0.10, 0.05]
    
    ### Calcul des scores : colonnes numériques
    score += score_range(phone['rating'], [(85, 1), (80, 0.5)]) * weights[0]
    score += score_range(phone['num_cores'], [(8, 1), (6, 0.5)]) * weights[1]
    score += score_range(phone['processor_speed'], [(2.8, 1), (2.4, 0.5)]) * weights[2]
    score += score_range(phone['battery_capacity'], [(4500, 1), (3500, 0.5)]) * weights[3]
    score += score_range(phone['ram_capacity'], [(12, 1), (8, 0.5)]) * weights[4]
    score += score_range(phone['internal_memory'], [(256, 1), (128, 0.5)]) * weights[5]
    score += score_range(phone['refresh_rate'], [(120, 1), (90, 0.5)]) * weights[6]
    score += score_range(phone['primary_camera_rear'], [(64, 1), (48, 0.5)]) * weights[7]
    
    ### Calcul des scores : colonnes booléennes
    score += score_boolean(phone['has_5g'], 1) * weights[-1]
    
    return score
        

Nous pouvons voir également que la colonne `rating` possède des valeurs manquantes qui sont cruciales pour le calcul du score de chaque téléphone. Pour remplacer ces valeurs, nous allons calculer les valeurs médianes des `rating` par marque de téléphones, et les attribuer aux téléphones qui n'ont pas de note répertoriée.

In [ ]:
### Calcul des médianes des `rating` par marque
median_per_brand = data.groupby('brand_name')['rating'].median()

data['rating'] = data.apply(
    lambda phone: median_per_brand[phone['brand_name']] if pd.isna(phone['rating']) else phone['rating'],
    axis=1
)

In [ ]:
### Fonction de catégorisation des téléphones
def determine_performance(phone) -> str:
    """
    Calcule un score de performance pour un téléphone donné 
    en fonction de ses caractéristiques techniques.

    Ce score permet ensuite de classifier le téléphone 
    en haut de gamme, milieu de gamme ou bas de gamme.
     
    :param phone: Prend en argument chaque téléphone (ligne) du dataset
    :return: Une chaîne de caractères qui indique la catégorie du téléphone
    """
    score: float = compute_total_score(phone)
    
    ### Attribution des catégories
    if score >= 0.7:
        return "haut performance"
    elif score >= 0.4:
        return "performance intermédiaire"
    else:
        return "basique"

In [ ]:
### Application au dataset
data['performance category'] = data.apply(determine_performance, axis=1)

data

**Estimation du rapport qualité/prix**

Pour prendre en compte l'avis des utilisateurs sur les téléphones par rapport au prix de vente qu'ils ont, nous allons créer une nouvelle métrique `quality-price ratio`. Nous avons vu que pour une même note, par exemple 85, les prix peuvent varier de moins de `100 €` à plus de `1000 €`, ce qui montre que la note ne reflète pas uniquement le prix, mais elle peut aussi intégrer la qualité perçue, la marque, la fiabilité, ou encore l’expérience utilisateur.

Cependant, pour définir cette métrique, nous allons l'appliquer par segment de gamme. C'est-à-dire qu'un téléphone de haute gamme peut avoir, avec cette nouvelle métrique, un ratio qui peut être bas. Et, nous devons noter que, les téléphones haut de gamme ne sont pas destinés à être économique, il n'est pas pertinent d'estimer ce rapport pour cette gamme de téléphone. Donc, pour ce faire, nous allons créer des ratios par gamme. 

Alors, nous allons utiliser la proposition suivante de segmentation des prix :
- Haut de gamme : > 700 €
- Milieu de gamme : 300 - 700 €
- Entrée de gamme : < 300 €

Nous allons créer une nouvelle colonne qui recense le segment du prix auquel appartient chaque téléphone.

In [ ]:
### Fonction de segmentation des prix
def segmenting_prices(price : float) -> str:
    """
    Détermine le segment dans lequel un téléphone se trouve à partir de son prix.
    
    :param price: Prix du téléphone
    :return: Segment auquel il appartient
    """
    if price < 300:
        return "bas"
    elif price <= 700:
        return "milieu"
    else:
        return "haut"

In [ ]:
### Application de la fonction de segmentation
data['price_segment'] = data['price (€)'].apply(segmenting_prices)

Nous allons maintenant calculer le ratio `rating / price`, mais étant donné qu'il n'est pas comparable entre les différents segments, nous allons ensuite normaliser le ratio dans chaque segment. Cela donne un score de 0 à 1, relatif à la performance du téléphone dans sa gamme de prix. Cela nous permettra d'avoir des choses comme, un téléphone à 200 € avec 80 de rating et un à 1000 € avec 90 de rating ayant des scores similaires dans leur segment respectif, car chacun est bon dans sa catégorie.

In [ ]:
### Calcul du ratio qualité / prix
data['ratio_rating_price'] = data['rating'] / data['price (€)']

In [ ]:
### Normalisation des ratios dans chaque segment de prix
data['ratio_segment'] = data.groupby('price_segment')['ratio_rating_price'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

data

Voici une petite analyse de la distribution du ratio segmenté :

* Segment bas (< 300 €) :
    * `Moyenne : 0.21`
    * `25% des modèles ont un score < 0.11`
    * `Score élevé (75e percentile) : > 0.27`
    * Les bons scores sont rares dans ce segment, les valeurs sont globalement faibles.

* Segment milieu (300–700 €) :
    * `Moyenne : 0.51`
    * `25% des modèles : < 0.28`
    * `75% des modèles : < 0.75`
    * Cette distribution est plus équilibrée, nous avons de bons segments pour identifier des rapports qualités-prix corrects.

* Segment haut (> 700 €) :
    * `Moyenne : 0.61`
    * `25% des modèles : < 0.46`
    * `75% des modèles : > 0.77`
    * Ici, nous avons plus de variance, et certains modèles haut de gamme sont bien notés pour leur prix.

Nous allons donc écrire une fonction nous permettant d'attribuer nos étiquettes `qualité/prix` selon des seuils basés sur l'analyse des distributions des différents segments.

In [ ]:
### Fonction d'étiquetage
def label_ratio(phone) -> str:
    """
    Fonction d'attribution de label sur le rapport qualité/prix en fonction du segment dans lequel se trouve le téléphone considéré.
    
    :param phone: Téléphone à étiqueter sur son rapport qualité/prix
    :return: Étiquette finale
    """
    
    ### Récupération des ratios
    val = phone['ratio_segment']
    segment = phone['price_segment']
    
    ### Disjonction de cas et attribution des labels
    ### Segment bas
    if segment == 'bas':
        if val >= 0.30:
            return "excellent"
        elif val >= 0.20:
            return "bon"
        elif val >= 0.10:
            return "moyen"
        else:
            return "faible"
        
    ### Segment milieu
    elif segment == 'milieu':
        if val >= 0.70:
            return "excellent"
        elif val >= 0.50:
            return "bon"
        elif val >= 0.30:
            return "moyen"
        else:
            return "faible"
    
    ### Segment haut
    elif segment == 'haut':
        if val >= 0.80:
            return "excellent"
        elif val >= 0.60:
            return "bon"
        elif val >= 0.40:
            return "moyen"
        else:
            return "faible"
    
    ### Renvoyer inconnu par mesure de sécurité (Possibles valeurs manquantes qui ne peuvent pas être remplacées)
    return "inconnu"

In [ ]:
### Application de la fonction d'étiquetage
data['quality-price_ratio'] = data.apply(label_ratio, axis=1)

data

**Enregistrement du dataframe**

Dans un premier temps, nous allons réorganiser les colonnes, et supprimer certaines dont nous n'aurons pas besoin pour la mise en place de notre modèle.

In [ ]:
### Nouvel ordre des colonnes
new_order = ['brand_name', 'model', 'price (€)', 'price_segment','rating', 'ratio_rating_price', 'ratio_segment', 'quality-price_ratio', 'has_5g', 'processor_brand', 'num_cores', 'processor_speed', 'battery_capacity', 'quality_battery_autonomy', 'fast_charging_available', 'fast_charging', 'ram_capacity', 'internal_memory', 'screen_size', 'refresh_rate', 'num_rear_cameras', 'num_front_cameras', 'os', 'primary_camera_rear', 'primary_camera_front', 'extended_memory_available', 'extended_upto', 'resolution_width', 'resolution_height', 'performance category']

data = data[new_order]

**Extraction du dataset traîté**

In [ ]:
data.to_csv('../data/processed/Smartphones_cleaned_dataset_processed.csv', index=False)